In [ ]:
"""
Most advanced model-building pattern: Model subclassing

1. In the __init__() method, define the layers the model will use [all inputs concatenation = features, intermediate layers, and output layers]
2. In the call() method, define the forward pass of the model, reusing the layers previously created
3. Instantiate your subclass, and call it on data to create its weights

"""

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
num_departments = 4

In [3]:
class CustomerTicketModel(keras.Model):
  # Step 1
  def __init__(self, num_departments):
    super().__init__()
    self.concat_layer = layers.Concatenate()
    self.mixing_layer = layers.Dense(64, activation = "relu")
    self.priority_scorer = layers.Dense(1, activation = "sigmoid")
    self.department_classifier = layers.Dense(num_departments, activation = "softmax")

  # Step 2
  def call(self, inputs):
    title = inputs["title"]
    text_body = inputs["text"]
    tags = inputs["tags"]

    features = self.concat_layer([title, text_body, tags])
    features = self.mixing_layer(features)

    priority = self.priority_scorer(features)
    department = self.department_classifier(features)

    return priority, department

In [5]:
import numpy as np
vocabulary_size = 1000
num_samples = 1280
num_tags = 100

title_data = np.random.randint(0, 2, size = (num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size = (num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size = (num_samples, num_tags))

In [10]:
np.max(title_data)

np.int64(1)

In [12]:
np.max(text_body_data)

np.int64(1)

In [13]:
np.max(tags_data)

np.int64(1)

In [4]:
# Step 3
model = CustomerTicketModel(num_departments=4)
priority, department = model({"title": title_data, "text": text_body_data, "tags": tags_data})

In [15]:
# Definition, compile, fit, evaluate, predict
model.compile(optimizer = "rmsprop",
              loss = ["mean_squared_error", "categorical_crossentropy"],
              metrics = [["mean_absolute_error"], ["accuracy"]])
# Structure of what is passed in loss and metrics should match exactly what gets returned by call

priority_data = np.random.random(size = (num_samples, 1))
department_data = np.random.randint(0, 2, size = (num_samples, num_departments))

In [16]:
np.max(priority_data)

np.float64(0.9998539727711167)

In [17]:
np.max(department_data)

np.int64(1)

In [18]:
model.fit({ "title": title_data, "text": text_body_data, "tags": tags_data} ,
          [priority_data, department_data], epochs = 1)

model.evaluate({ "title": title_data, "text": text_body_data, "tags": tags_data} ,
          [priority_data, department_data])

priority, department = model.predict({ "title": title_data, "text": text_body_data, "tags": tags_data})
print(f"Priority shape: {priority.shape}")
print(f"Department shape: {department.shape}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2927 - categorical_crossentropy_loss: 4.6720 - loss: 4.8083 - mean_absolute_error: 0.3034 - mean_squared_error_loss: 0.1363
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5125 - categorical_crossentropy_loss: 3.3579 - loss: 3.4602 - mean_absolute_error: 0.2711 - mean_squared_error_loss: 0.1023
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Priority shape: (1280, 1)
Department shape: (1280, 4)


In [19]:
priority[0]

array([0.55934906], dtype=float32)

In [20]:
priority_data[0]

array([0.9428608])

In [21]:
department[0]

array([0.21558753, 0.5280482 , 0.11206999, 0.14429428], dtype=float32)

In [22]:
np.argmax(department[0])

np.int64(1)

In [23]:
department_data[1]

array([1, 1, 0, 1])

In [24]:
np.argmax(department_data[1])

np.int64(0)

In [ ]:
"""
Freedom yet it comes at a cost, potential error surface is much larger

Model subclassing is not a graph based structure and so it is not possible to use plot_model and summary of shapes and topology.
Once thje model is instantiated, its forward pass becomes a complete black box.

"""

In [29]:
# mixing and matching different components
# Creating a Functional model that uses a subclassed model
class Classifier(keras.Model):

  def __init__(self, num_classes = 2):
    super().__init__()
    if num_classes == 2:
      num_units = 1
      activation = "sigmoid"

    else:
      num_units = num_classes
      activation = "softmax"
    self.dense = layers.Dense(num_units, activation = activation)


  def call(self, inputs):
    return self.dense(inputs)

inputs = keras.Input(shape = (3,))
features = layers.Dense(64, activation = "relu")(inputs)
outputs = Classifier(num_classes = 10) (features)
model = keras.Model(inputs = inputs, outputs = outputs)

In [30]:
# Creating a Subclass model that uses a Functional model

inputs = keras.Input(shape = (64,))
outputs = layers.Dense(1, activation = "sigmoid")(inputs)
binary_classifer = keras.Model(inputs = inputs, outputs = outputs)

class MyModel(keras.Model):
  def __init__(self, num_classes = 2):
    super().__init__()
    self.dense = layers.Dense(64, activation = "relu")
    self.classifier = binary_classifer

  def call(self, inputs):
    features = self.dense(inputs)
    return self.classifier(features)

model = MyModel()